# Notebook to estimate performance of a DRT system for many different parameter constellations
- estimations bases on previous simulation of the shortest distance to connect n random points (in any order) within unit circle
- formula when drawing n (number of passengers) out z random bus stops (with equal probability) 

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import datetime


# 3D
from mpl_toolkits.mplot3d import Axes3D 

# monitor memory space utility
import os, psutil # os not needed


In [2]:
# possible params:
n_stops_in_net = 2 # in h
n_pax = 12 # number of pax per tour OR average number of accepted request per hour OR average vehicle occupancy in h, pax leaves vehicle after travelling 10km
direct_trip_length = 8 # distance each passengers travels on average in kmh until leaving vehicle
stop_time_first_pax = 0.004165 # stop time when 1 passenger is boarding or alighting in h
stop_time_extra_pax = 0.001666 # stop time increase for each addtional passenger boarding or alighting in h
velocity = 30.0 # in km/h
n_vehicles = 1
vehicle_capacity = 8
vehicle_km_per_day = 160
maximum_tour_duration = 4.5# pausezeiten?
break_duration = 0.75 #

In [90]:
# Function to apply on scenario parameters


def vehicle_cost_calc(vehicle_capacity:int=vehicle_capacity, vehicle_km_per_day:float=vehicle_km_per_day)->tuple:
    """
    return tuple of 
        - total cost
        - dict of
            - vehicle fix cost per day in eur, 
            - vehicle km based cost in eur, 
            - personal cost per day in eur
            - personal cost share
    Inputs:
        - vehicle_capacity
        - km per day
    """
    fix_cost=(6+.042*vehicle_capacity*1.05**vehicle_capacity**.8) * 1000 / 365
    km_cost=(13+.15*vehicle_capacity*1.05**vehicle_capacity**.8 **.8) * 1000 * (vehicle_km_per_day / 75000)
    staff_cost=33.6 * 1000 / 365
    total_cost = fix_cost+km_cost+staff_cost
    # staff_cost_share should be: 0.48:26-38 seats, 0.39:39-55 seats, 0.38:26-38 seats # Tabelle 1: Kostenstrukturen spanischer Fernbusunternehmen in Abhängigkeit der Busgröße [MiFo13, S. 8, 12, 16] 
    return (total_cost, {'fix_cost':fix_cost, 'km_cost': km_cost, 'staff_cost':staff_cost, 'staff_cost_share':staff_cost/total_cost, 'marginal_cost_per_km':km_cost/vehicle_km_per_day})
#

staff_cost_share = vehicle_cost_calc()[1]['staff_cost_share']
#

def n_stops_on_path(n_pax:int=n_pax, n_stops_in_net:int=n_stops_in_net, stop_demand_fun=None) -> float:
    """
    returns expected number of unique stops rounded up to nearest int
    Inputs:
        - n_pax: number of individual passengers
        - n_stops_in_net: number of stops where pax could potentially picked up / dropped off 
    """
    # def ceil(f): return int(f//1+(f%1>0))
    # return ceil(n_stops_in_net*(1-((n_stops_in_net-1)/n_stops_in_net)**(2*n_pax)))
    return n_stops_in_net*(1-((n_stops_in_net-1)/n_stops_in_net)**(2*n_pax))
    # potentially let demand at each stop be drawn from distribution e.g. uniform(0-1)
#

def dist_total(n_stops_on_path:int=n_stops_on_path()) -> float:
    """
    return expected total travel distance
    Inputs:
        - n_stops_on_path: number of unique stops on path
    
    Function is an approximation to the resultion route length with respect of number of random points to connect within unit circle
    """
    return (1.2*(n_stops_on_path-1)/(n_stops_on_path**.5) + 0.1*n_stops_on_path - 0.0037*n_stops_on_path**1.7)*direct_trip_length
    #7*n_stops_on_path**0.2-0.05*n_stops_on_path+0.15*n_stops_on_path*0.99**n_stops_on_path-25
#

def stop_time_total(
        n_pax:int=n_pax, 
        n_stops_on_path:int=n_stops_on_path(), 
        stop_time_first_pax:float=stop_time_first_pax, 
        stop_time_extra_pax:float=stop_time_extra_pax
        ) -> float:
    """
    returns total stop time
    Inputs:
        - n_pax: number of individual passengers
        - n_stops_in_net: number of stops where pax could potentially picked up / dropped off 
    """
    return stop_time_first_pax*n_stops_on_path + stop_time_extra_pax*(2*n_pax-n_stops_on_path)
#

def share_driving_time(
        velocity:float=velocity,
        dist_total=dist_total(n_stops_on_path=n_stops_on_path()),
        stop_time_total=stop_time_total()
        ) -> float:
    """
    returns the share of time the vehicle is moving
    Inputs:
        - velocity: vehicle velocity when travelling
        - dist_total: total travel distance travelled to serve all pax
        - stop_time_total: total time stopping for boarding and alighting pax
    """
    return velocity*(dist_total/velocity)/(dist_total/velocity+stop_time_total)
#

def average_travel_speed(
        velocity:float=velocity,
        dist_total=dist_total(n_stops_on_path=n_stops_on_path()),
        stop_time_total=stop_time_total()
        ) -> float:
    """
    returns the average travel speed of vehicle per operating hour
    Inputs:
        - velocity: vehicle velocity when travelling
        - dist_total: total travel distance travelled to serve all pax
        - stop_time_total: total time stopping for boarding and alighting pax
    """
    return dist_total/(dist_total/velocity+stop_time_total)
#

def pax_served_per_hour(
        n_pax:int=n_pax, 
        velocity:float=velocity,
        dist_total=dist_total(n_stops_on_path=n_stops_on_path()),
        stop_time_total=stop_time_total()
        ) -> float:
    """
    returns the average travel speed of vehicle per operating hour
    Inputs:
        - velocity: vehicle velocity when travelling
        - dist_total: total travel distance travelled to serve all pax
        - stop_time_total: total time stopping for boarding and alighting pax
    """
    return n_pax*1/(dist_total/velocity+stop_time_total)
#
    

In [91]:
# label dicts for var and functions

# ['n_stops_on_path', 'dist_total', 'stop_time_total', 'share_driving_time', 'average_travel_speed', 'pax_served_per_hour']
outputLabelsDict = {
    'n_stops_on_path': 'Expeted number of unique stops',
    'dist_total': 'Total length of shortest path connecting all points',
    'stop_time_total': 'Total bus time stop time',
    'share_driving_time': 'Share of driving time of total bus operating time',
    'average_travel_speed': 'Average speed of bus during operating time',
    'pax_served_per_hour': 'Passengers served per hour',
}

# ['n_pax', 'n_stops_in_net', 'direct_trip_length', 'stop_time_first_pax', 'stop_time_extra_pax', 'velocity', 'stop_demand_fun']
paramLabelsDict = {
    'n_pax': 'Number of Passengers',
    'n_stops_in_net': 'Number of unique stops in (sub-)network',
    'direct_trip_length': 'Length of direct OD-path',
    'stop_time_first_pax': 'Stop time for single passenger enter/exit',
    'stop_time_extra_pax': 'Additional stop time for each addtional passenger enter/exit',
    'velocity': 'Free floating speed of vehicle',
    'stop_demand_fun': 'Function of demand distribution among stops',
}

In [92]:
# helper functions

def flattenList(l, degree:int=1):
    """
    Helper function to flat list degree times
    """
    if n_iters == 0: return(l)
    return [item for sublist in flattenList(l, degree-1) for item in sublist]
#

def listProduct(ar_list):
    """
    Helper Function to create all possible combination of multiple lists
    """
    if not ar_list:
        yield ()
    else:
        for a in ar_list[0]:
            for prod in listProduct(ar_list[1:]):
                yield (a,)+prod
#

def multi_indexer_assigner(d:dict, indexer_arr:list, value):
        if len(indexer_arr)<2:
            d[indexer_arr[0]]=value
            return
        multi_indexer_assigner(d[indexer_arr.pop(0)], indexer_arr, value)
#

def multi_indexer(d_arr:dict, indexer_arr:list):
    """
    iterates through indexes and return final element in d_arr
    """
    return(d_arr[indexer_arr[0]] if len(indexer_arr)<2 else multi_indexer_assigner(d[indexer_arr[0]], indexer_arr[1:], value))
#

def param_items_to_str(item_list:list):
    return ' '.join([str(key)+':'+str(vals[0] if type(vals)==list else vals) for (key,vals) in item_list])
#

def scale_increaser(n:float, scale_items:list):
    for scale_factor, scale_unit in scale_items:
        if n<scale_factor:
            break 
        n = n/scale_factor
    return (n, scale_unit)
#

def print_scenario_estimation(n_combinations, total_memory:int=psutil.Process().memory_info().rss):
    """
    prints a guess of computing resources needed to compute scenario
     
    Inputs:
        - n_combinations: number of scenario combinations
        - total_memory: in byte

    does not include fact that base params (e.g number of passengers) increase the scenarios possibilites by more than e.g. vehicle velocity
    """
    time_est = n_combinations*43.2/640000//1
    time_scale = [(60,'seconds'),(60,'minutes'),(24,'hours'),(7,'days'),(4.3,'weeks'),(12,'months'),(0,'years')]
    
    byte_est = 350*1e6/512000*n_combinations//1
    byte_scale = [(1e3, 'byte'),(1e3, 'kb'),(1e3, 'mb'),(1e3, 'gb'),(1e3, 'tb')]
    print("time_est, byte_est",time_est, byte_est)
    print( 'Number of scenario combinations:', n_combinations,
        '\nEstimated computing time:', scale_increaser(time_est, time_scale),
        '\nEstimated memory space usage:', scale_increaser(byte_est, byte_scale), ' equals' , byte_est/psutil.virtual_memory().total*1000//1/10, 'percent'
    )
#




In [93]:
def sci_params_generator(
        n_pax:list or int=[10], 
        n_stops_in_net:list or int=[100],
        direct_trip_length:list or float = [9.1],# distance each passengers travels on average in kmh until leaving vehicle
        stop_time_first_pax:list or float = [0.004165], # stop time when 1 passenger is boarding or alighting in h
        stop_time_extra_pax:list or float = [0.001666], # stop time increase for each addtional passenger boarding or alighting in h
        velocity:list or float = [30.0], # in km/h
        stop_demand_fun = [None],
)->dict:

        # ensure all params are provided within array
        param_name_to_sci_val = dict([(key, val if type(val)==list else [val]) for (key, val) in locals().items()])

        # estimation of computing times
        print_scenario_estimation(math.prod([len(x) for x in param_name_to_sci_val.values()]))

        return param_name_to_sci_val

In [94]:
def scenario_controller(
        run:list or bool=True,
        n_iterations: int=1,
        create_plots: bool=False,
        param_name_to_sci_val:dict=sci_params_generator(),
        kpi_funcs_ordered = [n_stops_on_path, dist_total, stop_time_total, share_driving_time, average_travel_speed, pax_served_per_hour]
    )->list:
    """
    calls different scenario aspects and stores different results
    Inputs:
        - run: bool or list. If False then only number of compinations for scenario is returned. If True all aspects and functions are run. Or vector strings with of function names that should be exececuted
        - n_iterations
        - create_plots
        - n_pax: ??number of pax per tour OR average number of accepted request per hour OR average vehicle occupancy in h
        - n_stops_in_net: number of stops where pax could potentially picked up / dropped off 
        - direct_trip_length: length of direct path between origin and demand of any pax
        - stop_time_first_pax: stop time when 1 passenger is boarding or alighting in h
        - stop_time_extra_pax: stop time increase for each addtional passenger boarding or alighting in h
        - velocity: vehicle velocity in km/h when driving

    Abbreivations: sci: scenarios, sce: scenario, param: modifiable scenario parameter, 

    kpi_name and kpis should be renamed to what they actually are: they compute key indicators for given cenario
    """
    # unpack parameter dictionary for easier later reference
    param_name_vals_sci, all_param_names = list(param_name_to_sci_val.items()), list(param_name_to_sci_val.keys())
    

    if not run: # early return when not running
        return

    # store list with names for later reference to avoid retrieving the name in nested loops
    kpi_names = [kpi.__name__ for kpi in kpi_funcs_ordered]
    # for each kpi determine recursively what its direct function inputs and what its influential factors are
    kpi_to_infl_factors, kpi_to_direct_func_inputs = {}, {}
    for kpi_name, kpi in zip(kpi_names, kpi_funcs_ordered):
        kpi_direct_func_inputs = list(kpi.__code__.co_varnames[:kpi.__code__.co_argcount])
        kpi_to_direct_func_inputs[kpi_name] = kpi_direct_func_inputs

        # flatten once
        kpi_influences_unsorted_duplicated = [item for sublist in 
                [[param_or_kpi_name] if param_or_kpi_name in all_param_names else kpi_to_infl_factors[param_or_kpi_name] for param_or_kpi_name in kpi_direct_func_inputs] 
            for item in sublist]
        kpi_to_infl_factors[kpi_name] = sorted(list(set(kpi_influences_unsorted_duplicated)), key=all_param_names.index)
    #

    kpi_params_arr = [(kpi_name, sorted(list(set(all_param_names) & set(kpi_to_infl_factors[kpi_name])), key=all_param_names.index)) for kpi_name in kpi_names]
    kpi_items_arr = [(kpi_name, [(param_name, param_name_to_sci_val[param_name]) for param_name in param_names]) for (kpi_name, param_names) in kpi_params_arr]

    # memory error is likely as the results for all the function are held in memory at the same time
    # holding results for function for which the outputs are needed for later functions is however necessary
    # a way to prevent this is to generate a functional form that directly return the results with respect to influence parameters    
    kpi_to_out = dict([(kpi_name, {
        # param only refers to influencial factor for kpi # maybe rename. 
        'param_names': [param_name for (param_name,param_vals) in params_items], 
        'param_items': [(param_name,param_vals) for (param_name,param_vals) in params_items], 
        'param_names_to_vals': dict([(param_name,param_vals) for (param_name,param_vals) in params_items]), 
        'params_dim':[len(param_vals) for (param_name,param_vals) in params_items], 
        'params_val_to_kpi_val': {}
        }) for (kpi_name, params_items) in kpi_items_arr])
    # res_arr is constructed from outside to inside res_arr[i] references param_name_vals_sci[flex_params_dims[0]][i]
    # res_arr is constructed from outside to inside res_arr[i][j] references param_name_vals_sci[flex_params_dims[0]][i] with param_name_vals_sci[flex_params_dims[1]][j]
    # res_arr is constructed from outside to inside res_arr[i][j][k] references param_name_vals_sci[flex_params_dims[0]][i] with param_name_vals_sci[flex_params_dims[1]][j] with param_name_vals_sci[flex_params_dims[1]][j][k] 
    
    
    # prepare all combinations to loop through # maybe there is a better way to do it.
    sci_params_flat = [[(param_name, param_val) for param_val in param_vals] for (param_name, param_vals) in param_name_vals_sci]
    # create all possible combs
    param_val_combs_sci = [dict(x) for x in listProduct(sci_params_flat)]

    
    # store interim results to quickly check whether functions needs to be applied or whether result is already computed. Allows for quick request of result
    infl_factors_vals_to_kpi_val = {}
    
    for sce in param_val_combs_sci:
        for kpi_name, kpi_fun in zip(kpi_names, kpi_funcs_ordered):
            # optain list of param_name, param_value pairs of given scenario that are influencial factor to current kpi
            sce_param_names_n_vals_infl_kpi = [(param_name, sce[param_name]) for param_name in kpi_to_out[kpi_name]['param_names']]
            # str used to store and retrieve interim result for a kpi for combination of influencial factor values
            param_str = param_items_to_str(sce_param_names_n_vals_infl_kpi)
            
            # if the current combination of values of influencial factor already used to compute current kpi, jump to next kpi
            if (kpi_name+'='+param_str) in infl_factors_vals_to_kpi_val.keys():
                continue
            
            # get input parameter values for kpi function
            # values are either simply taken from sce parameter or 
            # if they refere to a kpi that has already been computed, then the result needs to be looked up according to current sce params that influence the precomputed kpi  
            kpi_func_direct_inputs = [
                sce[param_or_kpi_name]
                    if param_or_kpi_name in all_param_names else 
                infl_factors_vals_to_kpi_val[
                    (param_or_kpi_name+'='+param_items_to_str([(param_name, sce[param_name]) for param_name in kpi_to_out[param_or_kpi_name]['param_names']]))
                ] 
                for param_or_kpi_name in kpi_to_direct_func_inputs[kpi_name]
            ]
            # unpack input parameters to execute function and optain kpi value
            kpi_val = kpi_fun(*kpi_func_direct_inputs)
            # store as intermediate result
            infl_factors_vals_to_kpi_val[(kpi_name+'='+param_str)] = kpi_val
            # store in final result dictonary
            kpi_to_out[kpi_name]['params_val_to_kpi_val'][param_str] = kpi_val
        #
    #
    
    # return dictionary with results
    return {
        'kpi_to_out': kpi_to_out, 
        # 'param_name_vals_sci': param_name_vals_sci, 
        'param_name_to_sci_val': param_name_to_sci_val,
        # 'param_val_combs_sci': param_val_combs_sci,
        }
#


time_est, byte_est 0.0 683.0
Number of scenario combinations: 1 
Estimated computing time: (0.0, 'seconds') 
Estimated memory space usage: (683.0, 'byte')  equals 0.0 percent


In [95]:
param_name_to_sci_val = sci_params_generator(
    n_pax=[int(x) for x in np.linspace(1,80,80)], #(1,80,80)
    n_stops_in_net=[int(x) for x in np.linspace(10, 200, 20)],#(10, 200, 20)
    velocity=[x for x in np.linspace(24, 50, 1)], #(24, 50, 13)
    direct_trip_length=[x for x in np.linspace(9.1, 9.1, 1)], #(6, 13.5, 6)
    stop_time_first_pax=[x for x in np.linspace(0.25, 1, 20)],#(0.25, 1, 20)
    stop_time_extra_pax= [x for x in np.linspace(2/60, 10/60, 4)]#(2/60, 10/60, 9)
    )

sci_kpis = scenario_controller(
    run=True, 
    param_name_to_sci_val=param_name_to_sci_val
)

# filename
path, filename, ext = './scenarios/', 'sc_04_29', '.npy'

time_est, byte_est 8.0 87500000.0
Number of scenario combinations: 128000 
Estimated computing time: (8.0, 'seconds') 
Estimated memory space usage: (87.5, 'mb')  equals 1.0 percent


In [96]:
sci_kpis['kpi_to_out']['n_stops_on_path']

{'param_names': ['n_pax', 'n_stops_in_net', 'stop_demand_fun'],
 'param_items': [('n_pax',
   [1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    31,
    32,
    33,
    34,
    35,
    36,
    37,
    38,
    39,
    40,
    41,
    42,
    43,
    44,
    45,
    46,
    47,
    48,
    49,
    50,
    51,
    52,
    53,
    54,
    55,
    56,
    57,
    58,
    59,
    60,
    61,
    62,
    63,
    64,
    65,
    66,
    67,
    68,
    69,
    70,
    71,
    72,
    73,
    74,
    75,
    76,
    77,
    78,
    79,
    80]),
  ('n_stops_in_net',
   [10,
    20,
    30,
    40,
    50,
    60,
    70,
    80,
    90,
    100,
    110,
    120,
    130,
    140,
    150,
    160,
    170,
    180,
    190,
    200]),
  ('stop_demand_fun', [None])],
 'param_names_to_vals': {'n_pax': [1,
   2,
   3,


In [97]:
# saving does not work

# # Save scenarios
# print('saving scenarios to:',path+filename+ext)
# np.save(path+filename+ext, sci_kpis)

# # load scenarios
# sci_kpis = np.load(path+filename+ext, allow_pickle=True)

In [98]:
def dims_to_params_generator(
        X='n_pax',
        Y=True,
        Z=None,
        L1=None,
        L2=None,
        C=None,
        M=None,
    ):
        """
        True or 1 output should be plotted here
        None not used
        """
        # what if same var on x and y? x and l1? X and C would not be a problem though.
        local_dict = locals()
        return local_dict

In [101]:
def rearrange_kpi_vals_to_dims(
        kpi_sci=sci_kpis['kpi_to_out'][list(sci_kpis['kpi_to_out'].keys())[0]], 
        param_name_to_vals_restr = None,
        dims_to_varnames=dims_to_params_generator()
    ):
    """
    function to extract data corresponding to the specified varnames
    should return data dict
        - scen_dict_restr: (potentially restricted) scenario_dict with param values
    """
    # kpi val dictionary
    params_val_to_kpi_val = kpi_sci['params_val_to_kpi_val']

    # if not specified otherwise in param_name_to_vals_restr full parameter space is used
    param_names_to_vals = param_name_to_vals_restr or kpi_sci['param_names_to_vals']
    print("param_names_to_vals",param_names_to_vals)
    # for later reference
    param_names = param_names_to_vals.keys()
    

    for (param_name, param_vals) in [(param_name, param_vals) for (param_name, param_vals) in param_names_to_vals.items() if not param_name in dims_to_varnames.values() and len(param_vals)>1]:
        print('WARNING: param '+ param_name +' is misspecified. Multiple parameter values provided but not assigned to any axis.', param_vals, 'will be reduced to', param_vals[:1])
        # enforces passive parameters to be fixed
        param_names_to_vals[param_name] = param_vals[:1]
    #

    # selects only those parameters that are relevant for plotting, i.e. that have a dimension specified to them
    dim_to_active_param_name_vals = dict([(dim_name, (param_name, param_names_to_vals[param_name])) for (dim_name, param_name) in dims_to_varnames.items() if param_name in param_names])

    # To Do check what to do with duplicates (same param for x and color)
    # check whether the order of dim_to_active_param_name_vals.items() is optimal or whether it should be sorted differently
    param_items_active = [item for (dim_key, item) in dim_to_active_param_name_vals.items()]
    param_items_active_long = [[(key, val) for val in vals] for (key, vals) in param_items_active]
    param_items_active_indexes_long = [[(key, i) for i in range(len(vals))] for (key, vals) in param_items_active]
    
    passive_param_dict = dict([(param_name, param_vals) for (param_name, param_vals) in param_names_to_vals.items() if not param_name in dims_to_varnames.values()])
    
    # needs to contain all active combinations and fixed combinations
    # assume that only params specified for plot have variance. the other are list of length 1 and we select only entry

    # create all possible combinations
    param_items_active_combinations = [dict(x) for x in listProduct(param_items_active_long)]
    param_items_active_combinations_indexes = [[i for (key,i) in arr] for arr in listProduct(param_items_active_indexes_long)]
    print("param_items_active_combinations",param_items_active_combinations)
    print("param_items_active_combinations_indexes",param_items_active_combinations_indexes)
    
    # create empty frame for kpi_arr
    kpi_arr = np.zeros([len(vals) for (key, vals) in param_items_active])

    for active_params_to_val_sce, scen_indexer in zip(param_items_active_combinations, param_items_active_combinations_indexes):
        # add params that are not active to have full lookup key
        params_val_sce = [(param_name, active_params_to_val_sce[param_name] if param_name in active_params_to_val_sce.keys() else passive_param_dict[param_name]) for param_name in param_names]
        # lookup_key
        fetch_str = param_items_to_str(params_val_sce)
        # select data from dict
        fetched_res = params_val_to_kpi_val[fetch_str]
        # assign to kpi_arr array
        multi_indexer_assigner(kpi_arr, scen_indexer, fetched_res)
    #
    
    # maybe the matrix needs to be transformed
    # extract output data from dict such that new np x dim array is filled
    # does it matter whether output is on X or Y for datastructure?
    # or whether input is on color or l1
    # should 1-dimensional inputs be ignored?
    # to-do: this is not yet correct: dims of vars need to be processed e.g. in mesh and for different lines and so.
    

    dim_to_vals = dict([(dim_key, param_vals) for (dim_key, (param_name, param_vals)) in dim_to_active_param_name_vals.items()])
    dim_to_kpi = dict([(dim_key, kpi_arr) for (dim_key, param_name) in dims_to_varnames.items() if param_name == True])
    # print("output_vars",dim_to_kpi)
    dim_to_vals.update(dim_to_kpi)
    # print("dim_to_vals",dim_to_vals)
    return dim_to_vals
arranged_kpi = rearrange_kpi_vals_to_dims()

param_names_to_vals {'n_pax': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80], 'n_stops_in_net': [10], 'stop_demand_fun': [None]}
param_items_active_combinations [{'n_pax': 1}, {'n_pax': 2}, {'n_pax': 3}, {'n_pax': 4}, {'n_pax': 5}, {'n_pax': 6}, {'n_pax': 7}, {'n_pax': 8}, {'n_pax': 9}, {'n_pax': 10}, {'n_pax': 11}, {'n_pax': 12}, {'n_pax': 13}, {'n_pax': 14}, {'n_pax': 15}, {'n_pax': 16}, {'n_pax': 17}, {'n_pax': 18}, {'n_pax': 19}, {'n_pax': 20}, {'n_pax': 21}, {'n_pax': 22}, {'n_pax': 23}, {'n_pax': 24}, {'n_pax': 25}, {'n_pax': 26}, {'n_pax': 27}, {'n_pax': 28}, {'n_pax': 29}, {'n_pax': 30}, {'n_pax': 31}, {'n_pax': 32}, {'n_pax': 33}, {'n_pax': 34}, {'n_pax': 35}, {'n_pax': 36}, {'n_pax': 37}, {'n_pax': 38}, {'n_pax': 39

In [100]:
def scenario_plotter(scenarios:dict=scenarios):
    """
    produces outputs and plots from results
    Inputs:
        - scenarios: dict consisting of
            - scenario_res: array consisting of dicts consisting of 
                - dict s_p (scenario parameters) 
                - dict r (results)
            - scenario_params dict consisting of
                - ...
    """

    
    outputs = list(scenarios['scenario_res'].keys())
    
    

    
    # varnamesWrapper()
    
    def plotMultiDimensional(scenarios:dict=scenarios, output:str=outputs[0], varnames:dict=varnamesWrapper() ,plottitle:str=None, contourlines:bool=False,expansionpath:bool=False, save:bool or str=False):
        """
        creates multidensional plot
        To-DO
            add possibility to fix one variable at specific value

        Inputs:
            - data: x dimensional np array
            - varnames varnames ordered for plotting: [X,Y,Z,L1,L2,M,C] where
                 - X: x-axis
                 - Y: y-axis
                 - Z: z-axis
                 - L1: variable to create lines for each
                 - L2: variable to create all line combinations with L1
                 - M: create a subplot for each value of that variable
                 - C: color variable
                 length of list must be fixed to 7. None value has to be provided if field should be left out specifc field should be empty, it should state None
        """

        
        # res_arr is constructed from outside to inside res_arr[i] references scenario_params[flex_param_names_dims[0]][i]
        # res_arr is constructed from outside to inside res_arr[i][j] references scenario_params[flex_params_dims[0]][i] with scenario_params[flex_params_dims[1]][j]
        # res_arr is constructed from outside to inside res_arr[i][j][k] references scenario_params[flex_params_dims[0]][i] with scenario_params[flex_params_dims[1]][j] with scenario_params[flex_params_dims[1]][j][k] 
        
        # what if same var on x and y? x and l1?
        # To-Do: clean varnames dict
        varnames = varnames
        
        extract_data_per_dim()
        
        return
        def getDataForVa2r(varname, scenarios=scenarios, output=output, varnames=varnames):
            """
            function to extract data corresponding to the specified varnames
            should return data dict
            """
            
            if varname==output:
                # can be affected by X,Y,Z,L1,L2,M but not C
                otherVars = list(set([varnames[key] for key in varnames.keys() if key != 'C' and not varnames[key] in [None, output]]))
                # 2do continue working here to make it more flexible!
                print(scenarios['scenario_res'][output])
                outputVar = [scenarios['scenario_res'][output]['res_arr'][i][0] for i in range(len(scenarios['scenario_params']['n_pax']))]
                return outputVar
            else:
                return scenarios['scenario_params'][varname]
        
        if None in [varnames['X'],varnames['Y'],varnames['Z']]:
            #plot 2D framework
            fig = plt.figure() 
            ax = fig.add_subplot(1, 1, 1)
            data_per_dim = extract_data_per_dim
            print('x,y',[(x,y) for (x,y) in zip(x,y)])
            ax.plot(x, y)
            ax.set_xlabel(varnames['X']) 
            ax.set_ylabel(varnames['Y']) 
            ax.set_title(plottitle or outputLabelsDict[output])
            fig.show()
            # plt.legend()
            # plt.show()
        else:
            print('PLOT 3D')
        if save: print('save', save)
        return
    
    return plotMultiDimensional()#('scenarioMatrix':scenarioMatrix, 'resPerVariable':resPerVariable)
    
resPerVariable = scenario_plotter()
resPerVariable

KeyError: 'scenario_res'

In [ ]:
# # plot 3D framework
# # import Axes3D 
# # domains 
# x = np.logspace(-1.,np.log10(5),50) # [0.1, 5] 
# y = np.linspace(6,9,50) # [6, 9] 
# z = np.linspace(-1,1,50) # [-1, 1] 

# # convert to 2d matrices 
# Z = np.outer(z.T, z) # 50x50 
# X, Y = np.meshgrid(x, y) # 50x50 

# # fourth dimention - colormap 
# # create colormap according to x-value (can use any 50x50 array) 
# color_dimension = X 

# # change to desired fourth dimension 
# minn, maxx = color_dimension.min(), color_dimension.max() 
# norm = matplotlib.colors.Normalize(minn, maxx) 
# m = plt.cm.ScalarMappable(norm=norm, cmap='jet') 
# m.set_array([]) 
# fcolors = m.to_rgba(color_dimension) 

# # plot 
# fig = plt.figure() 
# ax = fig.gca(projection='3d') 
# ax.plot_surface(X,Y,Z, rstride=1, cstride=1, facecolors=fcolors, vmin=minn, vmax=maxx, shade=False) 
# ax.set_xlabel('x') 
# ax.set_ylabel('y') 
# ax.set_zlabel('z') 
# fig.canvas.show()

In [ ]:
A, B, C, D = 2,3,2,2
dims = [A,B,C, D][:2]
m = np.zeros(dims)
for a in range(A):
    if len(dims)==1:
        m[a] = a*1000
    else:
        for b in range(B):
            if len(dims)==2:
                m[a][b] = a*1000+b*100
            else:
                for c in range(C):
                    if len(dims)==3:
                        m[a][b][c] = a*1000+b*100+c*10
                    else:
                        for d in range(D):
                            m[a][b][c][d] = a*1000+b*100+c*10+d
m

array([[   0.,  100.,  200.],
       [1000., 1100., 1200.]])

In [ ]:
x = np.logspace(-1.,np.log10(5),5) # [0.1, 5] 
y = np.linspace(6,9,5) # [6, 9] 
z = np.linspace(-1,1,5) # [-1, 1] 

# convert to 2d matrices 
Z = np.outer(z.T, z) # 50x50 
X, Y = np.meshgrid(x, y) # 50x50 

In [ ]:
X

array([[0.1       , 0.26591479, 0.70710678, 1.88030155, 5.        ],
       [0.1       , 0.26591479, 0.70710678, 1.88030155, 5.        ],
       [0.1       , 0.26591479, 0.70710678, 1.88030155, 5.        ],
       [0.1       , 0.26591479, 0.70710678, 1.88030155, 5.        ],
       [0.1       , 0.26591479, 0.70710678, 1.88030155, 5.        ]])

In [ ]:
Y

array([[6.  , 6.  , 6.  , 6.  , 6.  ],
       [6.75, 6.75, 6.75, 6.75, 6.75],
       [7.5 , 7.5 , 7.5 , 7.5 , 7.5 ],
       [8.25, 8.25, 8.25, 8.25, 8.25],
       [9.  , 9.  , 9.  , 9.  , 9.  ]])

In [ ]:
Z

array([[ 1.  ,  0.5 , -0.  , -0.5 , -1.  ],
       [ 0.5 ,  0.25, -0.  , -0.25, -0.5 ],
       [-0.  , -0.  ,  0.  ,  0.  ,  0.  ],
       [-0.5 , -0.25,  0.  ,  0.25,  0.5 ],
       [-1.  , -0.5 ,  0.  ,  0.5 ,  1.  ]])

In [ ]:
# Plots: 
# Add 21*demand proportion line to n_pax + n_stops_in_net . Add x for cottbus and berlin and line
# Or should some kind of density measure be applied?

# Plot types
# Better have outcome variable on axis instead of color?=more clearly visible
# =if continous: add lines to color gradient s.t. you have contour lines or add markers along lines
# You could try to make it continous for each plot: add in between points for each dimension based on some average function. Best=get continous function to describe data.

# 2D
# 2D+color
# 2D multiline(1var, color gradient)
# 2D multiline(2vars,color gradient ^ alpha or line type)
# 3D+color
# 3D+color × multiple figures
# 3D+color+mutliple surface (maybe with arrows helping to understand the direction)
# 3D+color+mutliple surface (maybe with arrows helping to understand the direction at edge of picture)
# Add expansion path that hold a metric fixed
# n_stops_on_path
# 2D + color: n_pax + n_stops_in_net
# total_dist
# 2D + color: n_pax + n_stops_in_net
# 2D: n_stops_on_path = total_dist
# total_stop_time:
# 2D: n_stops_on_path
# 2D + color: n_pax + n_stops_in_net
# 3D + color: n_pax + time_first_pax + time_extra_pax (given n_stops_in_net)
# total_stop_time / pax
# 2D + color: n_pax + n_stops_in_net
# 3D + color: n_pax + time_first_pax + time_extra_pax (given n_stops_in_net)
# average_operating_speed
# 2D + color: n_pax + n_stops_in_net
# 4D + color: n_pax + time_first_pax + time_extra_pax (given n_stops_in_net)
# 5D + color: n_pax + time_first_pax + time_extra_pax + velocity (given n_stops_in_net)
# pax_served_per_operating_hour
# 2D + color: n_pax + n_stops_in_net
# 3D + color: n_pax + n_stops_in_net
# 5D + color: n_pax + time_first_pax + time_extra_pax + velocity (given n_stops_in_net)
# Optain combinations leading to 12 (cottbus), 24, 36 =cost efficieny, 48, 60 =lower cost
# def pax_trip_time_average() result from above only not normalized to 1 hour? *(n_pax -1)/(n_pax)
# If order matters path has to be traversed 1-2 times without 1 connection
# 2D + color: n_pax + n_stops_in_net
# 5D + color: n_pax + time_first_pax + time_extra_pax + velocity (given n_stops_in_net)

In [ ]:
# 24wp/hour
# f_wp(n_wp)=x/60=/=x/60*(0.01+0.99^x)
# f_wp(1)=1/60=0.01666h 

# 36km/(24)=1.5km
# 60mins/24=2.5mins=0.04165h
# 0.04165h=0.01666h + 1.5km/(s km/h)
# 0.03 s km = 1.5 km
# s = 50 km/h = driving_speed
# operating_speed=36km/h 

# n_vehicles = 1
# vehicle_capacity = Inf
# personell_cost_share = 2/3 

# n_pax: (1,80,1)=80
# n_stops_in_net: (10,200,10)=20
# stop_time_first_pax: (20,60,2)=20
# stop_time_extra_pax: (2,10,2)=5
# velocity: (24,50,2)=13
# stop_demand_function: None, norm_stop=2
# direct_path_length: (6,13.5,1.5)=6
# centrality_param = (1,5,2)=3
# Total =80*20*20*5*13*2*6*3=74,880,000=8,650^2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import numpy as np


def heatmap2d(arr: np.ndarray):
    plt.imshow(arr, cmap='viridis')
    plt.colorbar()
    plt.show()


test_array = np.arange(100 * 100).reshape(100, 100)


In [ ]:
# a_v = np.linspace(1,4,31)
# b_v = np.linspace(0.985,.9995,31)
# mat = np.random.random((len(a_v), len(b_v)))

# for _i, a in enumerate(a_v):
#     for _j, b in enumerate(b_v):
#         mat[_i][_j]=min([(abs(fxab(x,a,b)), x) for x in range(2,100)])[0]
#         # mat[_i][(len(b_v)-_j)%len(b_v)]=_j*_i

# heatmap2d(mat)
